In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
from helper import *

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(700, 700, "Cubo", None, None)
glfw.make_context_current(window)

In [3]:
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

In [4]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas *Vertex* e *Fragment Shaders*

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando nosso código-fonte aos slots solicitados

In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o *Vertex Shader*
Se há algum erro em nosso programa *Vertex Shader*, nosso app para por aqui.

In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

In [9]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

In [11]:
cubes = []

offset = 0

for face in range(6):
    face = []
    for row in range(3):
        row = []
        for cube in range(3):
            row.append(offset)
            offset = offset + 24
            print(offset)
        face.append(row)
    cubes.append(face)


cube = Cube(cubes)

24
48
72
96
120
144
168
192
216
240
264
288
312
336
360
384
408
432
456
480
504
528
552
576
600
624
648
672
696
720
744
768
792
816
840
864
888
912
936
960
984
1008
1032
1056
1080
1104
1128
1152
1176
1200
1224
1248
1272
1296


# Modelagem do cubo

In [12]:
vertices_list = []

corner = get_corner_pieces()
center = get_center_pieces()
edge = get_edge_pieces()

# Face Cima
vertices_list.extend(corner[1])
vertices_list.extend(edge[0][1])
vertices_list.extend(corner[2])

vertices_list.extend(edge[2][1])
vertices_list.extend(center[4])
vertices_list.extend(edge[2][2])

vertices_list.extend(corner[5])
vertices_list.extend(edge[0][2])
vertices_list.extend(corner[6])

# Face Frente
vertices_list.extend(corner[1])
vertices_list.extend(edge[0][1])
vertices_list.extend(corner[2])

vertices_list.extend(edge[1][0])
vertices_list.extend(center[0])
vertices_list.extend(edge[1][3])

vertices_list.extend(corner[0])
vertices_list.extend(edge[0][0])
vertices_list.extend(corner[3])

# Face Direita
vertices_list.extend(corner[2])
vertices_list.extend(edge[2][2])
vertices_list.extend(corner[6])

vertices_list.extend(edge[1][3])
vertices_list.extend(center[3])
vertices_list.extend(edge[1][2])

vertices_list.extend(corner[3])
vertices_list.extend(edge[2][3])
vertices_list.extend(corner[7])

# Face Trás
vertices_list.extend(corner[5])
vertices_list.extend(edge[0][2])
vertices_list.extend(corner[6])

vertices_list.extend(edge[1][1])
vertices_list.extend(center[2])
vertices_list.extend(edge[1][2])

vertices_list.extend(corner[4])
vertices_list.extend(edge[0][3])
vertices_list.extend(corner[7])

# Face Esquerda
vertices_list.extend(corner[1])
vertices_list.extend(edge[2][1])
vertices_list.extend(corner[5])

vertices_list.extend(edge[1][0])
vertices_list.extend(center[1])
vertices_list.extend(edge[1][1])

vertices_list.extend(corner[0])
vertices_list.extend(edge[2][0])
vertices_list.extend(corner[4])

# Face Baixo
vertices_list.extend(corner[0])
vertices_list.extend(edge[0][0])
vertices_list.extend(corner[3])

vertices_list.extend(edge[2][0])
vertices_list.extend(center[5])
vertices_list.extend(edge[2][3])

vertices_list.extend(corner[4])
vertices_list.extend(edge[0][3])
vertices_list.extend(corner[7])

In [13]:
total_vertices = len(vertices_list)
vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
vertices['position'] = np.array(vertices_list)

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot

In [14]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Abaixo, enviamos todo o conteúdo da variável `vertices`
Veja os parâmetros da função `glBufferData` [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [15]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa `GLSL` (*Vertex Shaders*) com nossos dados
Primeiro, definimos o byte inicial e o offset dos dados.

In [16]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

Em seguida, soliciamos à GPU a localização da variável `position` (que guarda coordenadas dos nossos vértices). Definimos essa variável no *Vertex Shader*.

In [17]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

In [18]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

###  Pegamos a localização da variável `color` (`uniform`) do *Fragment Shader* para que possamos alterá-la em nosso laço da janela.

In [19]:
loc_color = glGetUniformLocation(program, "color")

### Nesse momento, exibimos a janela.

In [20]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a `OpenGL`.

Usaremos o `GL_TRIANGLE_STRIP` e modelaremos uma face do Cubo por vez, por questões didáticas. Iremos colorir cada face do Cubo com uma cor diferente.

In [21]:
from numpy import random
import math
d = 0.0
glEnable(GL_DEPTH_TEST)  # importante para 3D


def multiplica_matriz(a, b):
    m_a = a.reshape(4, 4)
    m_b = b.reshape(4, 4)
    m_c = np.dot(m_a, m_b)
    c = m_c.reshape(1, 16)
    return c


while not glfw.window_should_close(window):

    glfw.poll_events()

    # apenas para visualizarmos a esfera rotacionando
    d -= 0.005  # modifica o angulo de rotacao em cada iteracao
    cos_d = math.cos(d)
    sin_d = math.sin(d)

    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    glClearColor(0.1, 0.1, 0.1, 1.0)

    mat_rotation_z = np.array([cos_d, -sin_d, 0.0, 0.0,
                               sin_d,  cos_d, 0.0, 0.0,
                               0.0,      0.0, 1.0, 0.0,
                               0.0,      0.0, 0.0, 1.0], np.float32)

    mat_rotation_x = np.array([1.0,   0.0,    0.0, 0.0,
                               0.0, cos_d, -sin_d, 0.0,
                               0.0, sin_d,  cos_d, 0.0,
                               0.0,   0.0,    0.0, 1.0], np.float32)

    mat_rotation_y = np.array([cos_d,  0.0, sin_d, 0.0,
                               0.0,    1.0,   0.0, 0.0,
                               -sin_d, 0.0, cos_d, 0.0,
                               0.0,    0.0,   0.0, 1.0], np.float32)

    mat_transform2 = multiplica_matriz(mat_rotation_z, mat_rotation_x)
    mat_transform1 = multiplica_matriz(mat_rotation_y, mat_transform2)
    mat_transform = np.array([1,  0.0, 0, 0.0,
                              0.0,    1.0,   0.0, 0.0,
                              0, 0.0, 1, 0.0,
                              0.0,    0.0,   0.0, 1.0], np.float32)

    cube_index = 0

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform1)

    move = []

    for row in cube.cubes[0]:
        for c in row:
            move.append(c)

    draw = []

    i = 0
    for color, surface in zip(get_cube_colors(), cube.cubes[:6]):
        for rows in surface:
            for cub in rows:
                if cub not in move:
                    if (i == 0):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 20, 4)
                        draw.append((cub, 20))

                    elif (i == 1):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub, 4)
                        draw.append((cub, 0))

                    elif (i == 2):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 12, 4)
                        draw.append((cub, 12))

                    elif (i == 3):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 8, 4)
                        draw.append((cub, 8))

                    elif (i == 4):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 4, 4)
                        draw.append((cub, 4))

                    elif (i == 5):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 16, 4)
                        draw.append((cub, 16))

        i = i + 1

    for surface in cube.cubes:
        for rows in surface:
            for cub in rows:
                for offset in [0, 4, 8, 12, 16, 20]:
                    if (cub, offset) not in draw and cub not in move:
                        glUniform4f(loc_color, 0, 0, 0, 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + offset, 4)

    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform2)

    i = 0
    for color, surface in zip(get_cube_colors(), cube.cubes):
        for rows in surface:
            for cub in rows:
                if cub in move:

                    if (i == 0):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 20, 4)
                        draw.append((cub, 20))

                    elif (i == 1):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub, 4)
                        draw.append((cub, 0))

                    elif (i == 2):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 12, 4)
                        draw.append((cub, 12))

                    elif (i == 3):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 8, 4)
                        draw.append((cub, 8))

                    elif (i == 4):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 4, 4)
                        draw.append((cub, 4))

                    elif (i == 5):
                        glUniform4f(loc_color, color[0], color[1], color[2], 1)
                        glDrawArrays(GL_TRIANGLE_STRIP, cub + 16, 4)
                        draw.append((cub, 16))

        i = i + 1

        for surface in cube.cubes:
            for rows in surface:
                for cub in rows:
                    for offset in [0, 4, 8, 12, 16, 20]:
                        if (cub, offset) not in draw and cub in move:
                            glUniform4f(loc_color, 0, 0, 0, 1)
                            glDrawArrays(GL_TRIANGLE_STRIP, cub + offset, 4)

    glfw.swap_buffers(window)

glfw.terminate()


# Exercício
Modifique esse código para aplicar outras transformações (de sua escolha) conforme eventos do teclado.